In [26]:
from bs4 import BeautifulSoup
import requests
import uuid

# 한 건의 대화에 대한 정보를 담는 객체입니다.
class Conversation:
    # 질문(Question), 응답(Answer) 두 변수로 구성됩니다.
    def __init__(self, contentName, contentType, question, answer):
        if len(question) > 0 and question[0] == ':':
            question = question[1:]
        if len(answer) > 0 and answer[0] == ':':
            answer = answer[1:]
        while len(question) > 0 and question[0] == ' ':
            question = question[1:]
        while len(answer) > 0 and answer[0] == ':':
            answer = answer[1:]
        self.contentName = contentName
        self.contentType = contentType
        self.question = question
        self.answer = answer
        
    def __str__(self):
        return "질문: " + self.question + "\n답변: " + self.answer + "\n"

# 한 건의 영어 대화 주제 링크를 담는 객체입니다.
class Subject:
    def __init__(self, number, title, link):
        self.number = number
        self.title = title
        self.link = link
        
    def __str__(self):
        return "번호: " + self.number + "\n제목: " + self.title + "\n답변: " + self.link + "\n"
    
# 모든 영어 대화 주제를 추출하는 함수입니다.
def get_subjects(number):
    subjects = []
    
    req = requests.get('http://www.elllo.org/english/' + number)
    html = req.text
    
    soup = BeautifulSoup(html, 'html.parser')
    divs = soup.findAll('div', {"class": "mobilelist"})
    
    for subject in divs:
        tags = subject.findAll('a')
        
        for tag in tags:
            title = tag.text
            if title == '':
                continue
            link = tag['href']
            s = Subject(number, title, link)
            # print(s) # 추출된 주제를 출력합니다.
            subjects.append(s)
            
    return subjects

# 특정 대화 주제의 대화 내용을 크롤링합니다.
def get_conversation(title, number, url):
    conversations = []
    
    req = requests.get('http://www.elllo.org/english/' + number + "/" + url)
    html = req.text
    
    soup = BeautifulSoup(html, 'html.parser')
    if soup.text.find("transcript") != -1: # 스크립트 자체가 페이지에 없으면 취소
        return conversations
    if len(soup.findAll('div', {"class": "transcript"})) < 1:
        return conversations
    sayings = soup.findAll('div', {"class": "transcript"})[0]
    strongs = sayings.findAll('strong') # strong에 들어간 사람 이름 정보 확인

    idx = 0
    while True:
        if idx + 1 >= len(strongs):
            break
        question = strongs[idx].nextSibling # 사람 이름 다음에 메시지가 등장
        answer = strongs[idx + 1].nextSibling
        idx = idx + 1
        if question == None or answer == None:
            continue
        c = Conversation(number + str(uuid.uuid4()), idx, str(question), str(answer))
        print(c) # 추출된 대화 내용을 출력합니다.
        conversations.append(c)
        
    return conversations

In [27]:
subjects = []

# 모든 영어 대화 주제 링크를 얻어냅니다.
for i in range(0, 10):
    print("(" + str(i) + " / " + str(9) + ")")
    subjects.extend(get_subjects(str(1001 + (i * 50))))

print("전체 대화 주제의 개수는 " + str(len(subjects)) + "개 입니다.")

(0 / 9)
(1 / 9)
(2 / 9)
(3 / 9)
(4 / 9)
(5 / 9)
(6 / 9)
(7 / 9)
(8 / 9)
(9 / 9)
전체 대화 주제의 개수는 424개 입니다.


In [28]:
conversations = []

idx = 0
# 모든 대화 내용을 얻어냅니다.
for subject in subjects:
    print("(" + str(idx) + "/" + str(len(subjects)) + ") 현재 주제: " + subject.title + "\n")
    conversations.extend(get_conversation(subject.title, subject.number, subject.link))
    idx = idx + 1
    
print("전체 대화 주제의 개수는 " + str(len(conversations)) + "개 입니다.")

(0/424) 현재 주제: Future of Papers

질문: Taking 
답변: And actually I heard that they get more money from online advertising than from the print advertising so, yeah, I don't know really know why it matters come to think of it.

질문: And actually I heard that they get more money from online advertising than from the print advertising so, yeah, I don't know really know why it matters come to think of it.
답변:  I'm sure that the newspaper will 

질문: I'm sure that the newspaper will 
답변:  Yeah, yeah.

질문: Yeah, yeah.
답변:  But again, why is it that people will accept that but not books?

질문: But again, why is it that people will accept that but not books?
답변:  Cause I think ... I can handle a small article, like one page on screen, but I'm not for a long time. I can't read the same like. And also with news you just want to 

질문: Cause I think ... I can handle a small article, like one page on screen, but I'm not for a long time. I can't read the same like. And also with news you just want to 
답변: 

질문: So, Rachel, where are you from?
답변:  I'm from Scotland.

질문: I'm from Scotland.
답변:  Okay, Okay. Scotland like, what's the biggest city in Scotland would you say?

질문: Okay, Okay. Scotland like, what's the biggest city in Scotland would you say?
답변:  The capitol city is Edinburgh but the biggest city, size-wise and population

질문: The capitol city is Edinburgh but the biggest city, size-wise and population
답변:  Okay and you lived in ...? Did you live in one of those two cities?

질문: Okay and you lived in ...? Did you live in one of those two cities?
답변:  My grandparents lived in Edinburgh and I lived in Glasgow for quite awhile at university.

질문: My grandparents lived in Edinburgh and I lived in Glasgow for quite awhile at university.
답변:  Okay. I'm 

질문: Okay. I'm 
답변:  Yeah, pretty much.

질문: Yeah, pretty much.
답변:  Yeah, we both come from rainy climates.

질문: Yeah, we both come from rainy climates.
답변:  Yeah, the West Coast of Scotland gets more rain that the east coast, so it'

질문: Actually, I remember that one time you had a party and you made that really nice pumpkin dip.
답변:  Oh, yeah, that's a popular one. Everyone loves that when I make it.

질문: Oh, yeah, that's a popular one. Everyone loves that when I make it.
답변:  What do you need for it?

질문: What do you need for it?
답변:  It's really easy. All you need is one can of canned pumpkin, one eight ounce block of cream cheese, powdered sugar, and few spices.

질문: It's really easy. All you need is one can of canned pumpkin, one eight ounce block of cream cheese, powdered sugar, and few spices.
답변:  Is it easy to make?

질문: Is it easy to make?
답변:  It is. You just let the cream cheese melt a little, and then you blend it together with the pumpkin and the powdered sugar, and then after you get that really smooth and creamy, then you put in the nutmeg, ginger and cinnamon.

질문: It is. You just let the cream cheese melt a little, and then you blend it together with the pumpkin and the powdered sugar, and then af

질문: So, Ginger, I hear you have a new apartment.
답변:  Yes, it's great. I just moved in last week.

질문: Yes, it's great. I just moved in last week.
답변:  Nice. Where is it?

질문: Nice. Where is it?
답변:  It's near Victory Monument. Do you know where that is?

질문: It's near Victory Monument. Do you know where that is?
답변:  Yeah, I do. How do you get to work from there?

질문: Yeah, I do. How do you get to work from there?
답변:  Well, I take the Sky Train.

질문: Well, I take the Sky Train.
답변:  Oh, that's really convenient. It is. It is. It's just a five minute walk. And what's the neighborhood like?

질문: Oh, that's really convenient. It is. It is. It's just a five minute walk. And what's the neighborhood like?
답변:  It's a crowded neighborhood. There are a lot of people. Lots of cars. Just lots of energy in general.

질문: It's a crowded neighborhood. There are a lot of people. Lots of cars. Just lots of energy in general.
답변:  And is it 

질문: And is it 
답변:  It's very convenient. It's about a fiv

질문: So, what about Sri Lankan food? What are some of the good dishes in Sri Lanka?
답변:  What we eat every day, almost every day, is rice and curry, so lunch is usually rice and curry. Curry will be usually spicy, and we eat lots of veggies, so we'll have like at least two or three dishes of different kinds of veggies, all cooked with different kinds of spices, and we have so many styles, and we'll have something fried as well along with green leaves, a dish that's a definite Sri Lankan lunch.

질문: What we eat every day, almost every day, is rice and curry, so lunch is usually rice and curry. Curry will be usually spicy, and we eat lots of veggies, so we'll have like at least two or three dishes of different kinds of veggies, all cooked with different kinds of spices, and we have so many styles, and we'll have something fried as well along with green leaves, a dish that's a definite Sri Lankan lunch.
답변:  Is it really spicy?

질문: Is it really spicy?
답변:  The curry yes, but it depends, l

...

질문: So after Laos, where did you go?
답변:  After Laos, I flew to Vietnam. And actually I took the bus into Vietnam.
                              

질문: After Laos, I flew to Vietnam. And actually I took the bus into Vietnam.
                              
답변:  OK. 

질문: OK. 
답변:  Yes. And in Vietnam, I traveled from the north, from Hanoi, to Ho Chi Minh City. But because my time was so short, I could only spend one or two days in each city.

질문: Yes. And in Vietnam, I traveled from the north, from Hanoi, to Ho Chi Minh City. But because my time was so short, I could only spend one or two days in each city.
답변:  Ah! What city did you like the most?

질문: Ah! What city did you like the most?
답변:  I think I prefer Hanoi the most. The food, the street food is amazing. Itâs a very lively city, lots of motorcycles and traffic so it reminded me of Nigeria where I was born a little bit.

질문: I think I prefer Hanoi the most. The food, the street food is amazing. Itâs a very lively city, lots 

질문: Alright, hi! I'm Adam, and I'm with Jin today, and we're talking about your home country, Nepal.
답변:  Yes, that's right.

질문: Yes, that's right.
답변:  Very cool. I know that the Himalayas, and even Mt. Everest is in Nepal, which is amazing. What else can you tell us about the 

질문: Very cool. I know that the Himalayas, and even Mt. Everest is in Nepal, which is amazing. What else can you tell us about the 
답변:  Yeah, well actually, when people think about Nepal, they just talk about the mountains, but then there are actually three regions. There is the mountain region, and then there is another hilly region, and then Terai region, so then in Terai region, the earth, like the 

질문: Yeah, well actually, when people think about Nepal, they just talk about the mountains, but then there are actually three regions. There is the mountain region, and then there is another hilly region, and then Terai region, so then in Terai region, the earth, like the 
답변:  So in the south it's kind of the

질문: OK, we'll talk about movies.
답변:  OK.

질문: OK.
답변:  Have you ever seen a German movie?

질문: Have you ever seen a German movie?
답변:  I haven't seen a German movie. I haven't seen very many movies. 

질문: I haven't seen a German movie. I haven't seen very many movies. 
답변:  Oh, OK.

질문: Oh, OK.
답변:  Even English movies, I haven't seen very many.

질문: Even English movies, I haven't seen very many.
답변:  Well, have you seen a Canadian movie?

질문: Well, have you seen a Canadian movie?
답변:  Uh, I don't know. Have I?

질문: Uh, I don't know. Have I?
답변:  Well, 

질문: Well, 
답변:  I've definitely seen a Canadian actor.

질문: I've definitely seen a Canadian actor.
답변:  Yeah, all actors in America are Canadian

질문: Yeah, all actors in America are Canadian
답변:  I think so.

질문: I think so.
답변:  OK, have you ever bought something from Italy?

질문: OK, have you ever bought something from Italy?
답변:  From Italy. I have. I have bought - not for me - but I bought my mother a bag from Italy one time.

질문: 

질문: So Pernais, Jamaica's quite a small country, right? 
답변:  It is.

질문: It is.
답변:  What's the population?  Not exactly, just 

질문: What's the population?  Not exactly, just 
답변:  

질문: 
답변:  Okay, that's pretty small.  Do you have a problem with people leaving, like for university or of to get better jobs going to other countries?

질문: Okay, that's pretty small.  Do you have a problem with people leaving, like for university or of to get better jobs going to other countries?
답변:  Yes, a lot of people leave, but I don't know if it's so much a problem because Jamaicans have been 

질문: Yes, a lot of people leave, but I don't know if it's so much a problem because Jamaicans have been 
답변:  Half?

질문: Half?
답변:  Yes.

질문: Yes.
답변:  So that means there's like 5.4 million Jamaicans?

질문: So that means there's like 5.4 million Jamaicans?
답변:  Yes.

질문: Yes.
답변:  And only half of them live in Jamaica?

질문: And only half of them live in Jamaica?
답변:  Yes.  Possibly.

질문: Yes.  Possibly.
답변:  

질문: So Abidemi, you were a teacher in Canada, correct?
답변:  That's true Todd.

질문: That's true Todd.
답변:  Now, where did you teach?

질문: Now, where did you teach?
답변:  I taught in Northern Canada in a province called Nunavut.

질문: I taught in Northern Canada in a province called Nunavut.
답변:  Nunavut.

질문: Nunavut.
답변:  Nunavut.

질문: Nunavut.
답변:  Nunavut. Wow is it like a native Americans  type ..?

질문: Nunavut. Wow is it like a native Americans  type ..?
답변:  Yeah, and except we call them Inuits.

질문: Yeah, and except we call them Inuits.
답변:  Inuits?

질문: Inuits?
답변:  Aboriginals and First Nations.

질문: Aboriginals and First Nations.
답변:  So what was it like?

질문: So what was it like?
답변:  I really liked it. It was great. Very different from other parts of Canada. Very cold obviously because it's in the far north, and the communities are very isolated. Very few people live there. In the particular community were I lived, there were about a thousand people, maybe less living there. S

In [29]:
import xlsxwriter

# 엑셀 워크 북 및 워크 시트를 생성합니다.
workbook = xlsxwriter.Workbook('Conversation Data 1.xlsx')
worksheet = workbook.add_worksheet()

row = 0
col = 0

# 모든 대화 내용을 엑셀로 저장합니다.
for c in conversations:
    worksheet.write(row, col, c.contentName)
    worksheet.write(row, col + 1, c.contentType)
    worksheet.write(row, col + 2, c.question)
    worksheet.write(row, col + 3, c.answer)
    row = row + 1
    
workbook.close()

In [ ]:
# 내보내기 이후에 엑셀에서 공백이 있거나 None으로 처리된 것들은 제거합니다.